In [11]:
import torch
import torch.nn as nn
import math
from torch import Tensor
import traceback
import torch.optim as optim
from tqdm.notebook import tqdm
import string
import ast
import pandas as pd
import copy

device = "cuda" if torch.cuda.is_available() else "cpu"

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 512):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class SpellCorrectionModel(nn.Module):
    def __init__(self, word_vocab_size, char_vocab_size, char_embedding_dim=256, word_embedding_dim=768, word_n_head=8, char_n_head=4, word_ffw=786, char_ffw=256):
        super(SpellCorrectionModel, self).__init__()
        self.word_embedding_dim = word_embedding_dim
        self.word_embedding = nn.Embedding(word_vocab_size, word_embedding_dim, padding_idx=0)
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim, padding_idx=0)

        self.pos_encoder = PositionalEncoding(word_embedding_dim+char_embedding_dim, dropout=0.1)
        self.norm_w = nn.LayerNorm(word_embedding_dim+char_embedding_dim)
        self.norm_c = nn.LayerNorm(char_embedding_dim)
        word_transformer = nn.TransformerEncoderLayer(d_model=(word_embedding_dim+char_embedding_dim), nhead=word_n_head, dim_feedforward=(word_ffw+char_ffw), dropout=0.1, activation='gelu', batch_first=True)
        self.word_transformer_encoder = nn.TransformerEncoder(word_transformer, num_layers=12, norm=self.norm_w)
        char_transformer = nn.TransformerEncoderLayer(d_model=char_embedding_dim, nhead=char_n_head, dim_feedforward=char_ffw, dropout=0.1, activation='gelu', batch_first=True)
        self.char_transformer_encoder = nn.TransformerEncoder(char_transformer, num_layers=4, norm=self.norm_c)
        # self.linear_layer = nn.Linear(char_embedding_dim, 768)

        self.correction = nn.Linear(word_embedding_dim+char_embedding_dim, word_vocab_size)
        # self.softmax = nn.Softmax(dim=-1)
        self.sigmoid = nn.Sigmoid()
        self.is_correct = nn.Linear(word_embedding_dim+char_embedding_dim, 1)

    def forward(self, word_text, char_text):
        w = torch.tensor(word_text).to(device)
        c = torch.tensor(char_text).to(device)

        # print("Input : ",w.shape,c.shape)

        # assign mask for word and char
        padding_mask_w = (w != 0).float()

        # padding_mask_c = (torch.sum(c,dim=-1,keepdims=False) != 0).unsqueeze(2).float()
        # padding_mask_c = (c != 0).unsqueeze(3).float()

        # print("mask w : ",padding_mask_w.shape)
        # print("mask c : ",padding_mask_c.shape)

        word_embedded_text = self.word_embedding(w)
        char_embedded_text = self.char_embedding(c)

        char_embedded_text = torch.mean(char_embedded_text, dim=2, keepdims=False)
        char_embedded_text = self.char_transformer_encoder(char_embedded_text, src_key_padding_mask=padding_mask_w)
        linear_input = torch.cat((word_embedded_text, char_embedded_text), dim=2)
        src = self.pos_encoder(linear_input)
        output = self.word_transformer_encoder(src, src_key_padding_mask=padding_mask_w)
        output_spell = self.sigmoid(self.is_correct(output))
        output_correction = self.correction(output)
        # print(output_spell,output_correction)
        #print("Output position fix : ",output_correction.shape)
        
        return output_correction.cpu(), output_spell.cpu()


In [12]:
with open('merge_word_vocab_13126.txt','r') as f:
    word_tokenizer = f.read().split('\n')
word_tokenizer.insert(0,"<unk>")
word_tokenizer.insert(0,"<pad>")
word_tokenizer.insert(0,"<end>")
word_tokenizer.insert(0,"<start>")

map_word = {j:i for i,j in enumerate(word_tokenizer)}
# map_word

In [13]:
with open('char_vocab.txt','r') as f:
    char_tokenizer = f.read().split('\n')
char_tokenizer.insert(0,"<unk>")
char_tokenizer.insert(0,"<pad>")
char_tokenizer.insert(0,"<end>")
char_tokenizer.insert(0,"<start>")

map_char = {j:i for i,j in enumerate(char_tokenizer)}
# map_char

In [14]:
sample_text = ["Hôm nay trời thật đẹp","Chủ nhật này đi chơi không ?"]

def mapping_batch(sample):
    try:
        max_word_length = max([len(x.split(' ')) for x in sample])
        max_char_length = max([max([len(word) for word in sentence.split(' ')]) for sentence in sample])
    except:
        return "", "", 0
    # #print(max_word_length, max_char_length)
    res = []
    res_char = []
    for i in sample:
        out = []
        out_char = []
        for j in i.split(' '):
            out.append(map_word[j] if j in map_word.keys() else map_word['<unk>'])
            r = []
            for g in j:
                r.append(map_char[g] if g in map_char.keys() else map_char['<unk>'])
            while len(r) < max_char_length:
                r.append(2)
            out_char.append(r)
        while len(out) < max_word_length:
            out.append(2)
            out_char.append([2] * max_char_length)
            
        res.append(out)
        res_char.append(out_char)
    return res, res_char , max_word_length

# mapping_batch(sample_text)

In [15]:
df_test = pd.read_csv("test.csv")

In [47]:
# Instantiate your model and optimizer
loaded_model = SpellCorrectionModel(13130,214).to(device)
loaded_model.eval()
loaded_optimizer = optim.Adam(loaded_model.parameters(), lr=0.001)

# Load the saved model and optimizer state
checkpoint = torch.load('latest.pth')
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# If you need to continue training, set the model to training mode
# loaded_model.train()


In [51]:
sample = df_test.generate.values.tolist()[0:2]
sample

['Trong dài hạn , siegel cho biết : Nhữmg mối quan hệ này không thực sự ạ ra trái nnngọt .',
 'Hiện tại , harvard đanng nỗ lực sắp p lại cơ cấu tổ chức và danh mục đầu tư của quỹ hiến tặng.']

In [52]:
w, c, max_length = mapping_batch(sample)

In [53]:
with torch.no_grad():
    output_correction, output_spell = loaded_model(w, c)

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 23 but got size 0 for tensor number 1 in the list.

In [46]:
output_spell

tensor([[[0.0716],
         [0.0745],
         [0.0771],
         [0.0751],
         [0.0834],
         [0.0896],
         [0.0729],
         [0.0781],
         [0.0740],
         [0.0794],
         [0.0701],
         [0.0824],
         [0.0752],
         [0.0726],
         [0.0722],
         [0.0736],
         [0.0794],
         [0.0786],
         [0.0803],
         [0.0726],
         [0.0721]]])

In [76]:
df_test.head()

,text,generate,label
0,"Trong dài hạn , Siegel cho biết : Những mối qu...","Trong dài hạn , siegel cho biết : Nhữmg mối qu...","[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,"Hiện tại , Harvard đang nỗ lực sắp xếp lại cơ ...","Hiện tại , harvard đanng nỗ lực sắp p lại cơ c...","[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,Và việc rút vốn khỏi công ty của Mindich là mộ...,àV việc Rút vốn khỏi công ty của mindich là mộ...,"[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, ..."
3,"Giống như nhiều đối thủ khác , Harvard đã từ l...","Giống như nhiều đđđối thủ khác , harvard ã từ ...","[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, ..."
4,"Tuy nhiên , mặc cho những mối quan hệ tốt ở Ph...","Tuy nhiên , mặc cho nhg mối Quan hệ tốt ở phố ...","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, ..."


In [48]:
3 & 9

1

In [34]:
a = Solution().maximumLength([4,36,9,16,1,1,4,121,64,4])

[1, 1, 4, 4, 4, 9, 16, 36, 64, 121]
1 1 1
2 4 2
3 4 2
4 4 2


In [43]:
2 & 8

0

In [52]:
7 | 10

15

In [54]:
a = [15,10,1,14,9,4]
g = a[0]
for i in a[1:]:
    g = g | i

g

15

In [65]:
"0".isdigit()

True

In [101]:
import torch
import torch.nn as nn

# Assuming you have a binary classification problem
# Random predictions (logits) and binary labels
predictions = torch.randn(1, requires_grad=True)
labels = torch.randint(0, 2, (1,), dtype=torch.float32)

# Applying sigmoid activation to convert logits to probabilities
probabilities = torch.sigmoid(predictions)

# Calculating BCELoss
criterion = nn.BCELoss()
loss = criterion(probabilities, labels)

print("Predictions:", probabilities.item())
print("True Label:", labels.item())
print("BCE Loss:", loss.item())


Predictions: 0.9067719578742981
True Label: 0.0
BCE Loss: 2.372706651687622


In [1]:
import json

def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Load each line as a JSON object
            json_object = json.loads(line.strip())
            data.append(json_object)
    return data

# Replace 'example.jsonl' with the path to your JSONL file
file_path = '../synthetic_data_27_01_24.jsonl'
result = read_jsonl_file(file_path)

# Now 'result' contains a list of dictionaries, each representing a JSON object from the file
print(result[0])


{'id': 12450, 'passage': 'Vào ngày 1 tháng 9 năm 2013, họ phát hành "Strong", đạt vị trí thứ 16 trên UK Singles Chart.\xa0 Bài hát sau đó đã được sử dụng trong bộ phim truyền hình Mỹ năm 2014 "Reckless" cho tập thứ hai của nó, "Parting Shots".\xa0 Vào ngày 9 tháng 9 năm 2013, họ phát hành album phòng thu đầu tay If You Wait, đạt vị trí thứ hai trên Bảng xếp hạng Album của Vương quốc Anh.\xa0 Nó cũng đạt vị trí thứ hai trên Bảng xếp hạng Album của Úc, vị trí thứ 11 trên Bảng xếp hạng Album của Pháp, vị trí thứ 13 trên Bảng xếp hạng Album của Ireland và vị trí thứ 22 trên Bảng xếp hạng Album của New Zealand.\xa0 Ban nhạc được ký hợp đồng với Hãng đĩa Columbia tại Hoa Kỳ.Vào ngày 8 tháng 12 năm 2013, một đĩa đơn cho album, "Nightcall" được phát hành.\xa0 Bìa của họ đã được sử dụng một lần nữa cho chương trình "Reckless" trong tập cuối "Civil Wars phần 2".\nVào ngày 13 tháng 1 năm 2014, ban nhạc biểu diễn "Strong" và "Wasting My Young Years" vào Late Night với Jimmy Fallon,đánh dấu màn trì

In [2]:
import json



In [4]:
result[0]['id']

12450

In [54]:
import torch


a = torch.tensor([0.1,0.5,0.8,0.9])
torch.round(a)

tensor([0., 0., 1., 1.])